In [ ]:
import numpy as np
import random
import math

# Test both functions


def change_cnot_gate_to_one_qubit_gates(circuit_layer: list, gate_options: list, selected_qubit: int) -> list:

    """
    This function turns the control and target qubits into one-qubit gates

    """

    gate = circuit_layer[selected_qubit]
    gate_index = gate[5]
    if gate[0:4] == 'ctrl':
        target_index = circuit_layer.index('trgt_' + gate_index)
        circuit_layer[target_index] = random.choice(gate_options)
    elif gate[0:4] == 'trgt':
        control_index = circuit_layer.index('ctrl_' + gate_index)
        circuit_layer[control_index] = random.choice(gate_options)
    
    circuit_layer[selected_qubit] = random.choice(gate_options)
    
    return circuit_layer

def swap_cnot_gate(circuit_layer: list, selected_qubit: int):
    """
    This function swaps the control and target qubits in 
    the CNOT gate

    """

    gate = circuit_layer[selected_qubit]
    if gate[0:4] == 'ctrl': # testar esse if
        cnot_value = gate[5]
        target_index = circuit_layer.index('trgt_'+ cnot_value)
        circuit_layer[selected_qubit], circuit_layer[target_index] =  circuit_layer[target_index], circuit_layer[selected_qubit] 
    elif gate[0:4] == 'trgt':
        cnot_value = gate[5]
        control_index = circuit_layer.index('ctrl_' + cnot_value)
        circuit_layer[selected_qubit], circuit_layer[control_index] =  circuit_layer[control_index], circuit_layer[selected_qubit]
    
    return circuit_layer


def drop_layers(circuit):
    n_layers = random.randint(int(math.floor(len(circuit)/2)) - 1, len(circuit))
    selected_indexes = np.random.randint(0, high=len(circuit), size=n_layers)
    
    return [circuit[index] for index in selected_indexes]




class AnsatzMutation:

    """
    This Mutation Class randomly swaps Ansatz layers, changes the  
    the gates placed in certain positions

    """
    def __init__(self, mutation_rate: float):
        self.mutation_rate = mutation_rate
        self.one_qubit_gates = ['pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard']
        #self.gate_options_with_cnot = ['pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard']

    # Make it drop out a few layers during mutation to guarantee that it will remain shallow and diverse somehow
    def place_a_single_gate(self):
        return random.choice(self.one_qubit_gates) 
    
    def generate_partially_entangled_layer(self, layer):
        pairs = [(wire,wire+1) for wire in range(len(layer))]
        #print(pairs)
        selected_pair = random.choice(pairs)
        control, target = selected_pair
        
        layer = ['ctrl_0' if wire == control else
                'trgt_0' if wire == target else
                random.choice(self.one_qubit_gates)
                for wire in range(len(layer))]
    
        return layer
    
    def shuffle_layers(self, circuit):
        return random.sample(circuit, k=len(circuit))

    def _do(self, offspring):
        mutated_offspring = []
        for individual in offspring:
            mutated_individual = []
            random_value = np.random.random()
            if random_value < self.mutation_rate:
                choice = np.random.randint(0, 2)
                # Randomly make the circuit ansatz drop a few layers or shuffle them
                if choice == 0:
                    random_mutation_action = np.random.randint(0,2)
                    if random_mutation_action == 0:
                        mutated_offspring.append(drop_layers(individual))
                    elif random_mutation_action == 1:
                        mutated_offspring.append(self.shuffle_layers(individual))
                elif choice == 1:
                    # Or make it change entanglement patterns, turn CNOTs into one qubit gates, or simply change gates
                    for circuit_layer in individual:
                            random_qubit_position = np.random.randint(0, len(circuit_layer))
                            random_gate = circuit_layer[random_qubit_position]
                            circuit_layer[random_qubit_position]
                            if random_gate[0:4] == 'ctrl' or random_gate[0:4] == 'trgt': 
                                random_mutation_action = np.random.randint(0,2)
                                random_qubit_position = np.random.randint(0, len(circuit_layer))
                                if random_mutation_action == 0:
                                    mutated_individual.append(swap_cnot_gate(circuit_layer, random_qubit_position))
                                elif random_mutation_action == 1:
                                    mutated_individual.append(change_cnot_gate_to_one_qubit_gates(circuit_layer, self.one_qubit_gates, random_qubit_position))
                            else:
                                new_layer = circuit_layer
                                random_mutation_action = np.random.randint(0,2)
                                if random_mutation_action == 0:
                                    new_layer[random_qubit_position] = random.choice(self.one_qubit_gates)
                                    mutated_individual.append(new_layer)
                                else:
                                    # Besides mutating entanglement patterns and changing cnots into one qubit gates, we could add partial and full entaglement by mutation in a few layers
                                    # this would synthetize more rich and diverse quantum circuits over the iterations :)
                                    mutated_individual.append(self.generate_partially_entangled_layer(new_layer))
                                
                    mutated_offspring.append(mutated_individual)        
            else:
                mutated_offspring.append(individual)           
                

        offspring_a, offspring_b = mutated_offspring
        return offspring_a, offspring_b

In [ ]:
mutate = AnsatzMutation(0.1)

In [53]:
population = [[['rz_gate', 'rz_gate', 'phase', 'ry_gate'], ['hadamard', 'pauli_y', 'hadamard', 'pauli_z'], ['rz_gate', 'hadamard', 'hadamard', 'hadamard'], ['rx_gate', 'ctrl_0', 'trgt_0', 't']], [['pauli_z', 'ctrl_0', 'trgt_0', 'pauli_y'], ['ctrl_0', 'trgt_0', 'ry_gate', 'pauli_z'], ['t', 'pauli_z', 'pauli_y', 'pauli_x'], ['hadamard', 't', 'pauli_x', 'pauli_z'], ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], ['t', 'ctrl_0', 'trgt_0', 'rx_gate'], ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], ['phase', 'ctrl_0', 'trgt_0', 'rz_gate'], ['rz_gate', 'rx_gate', 'rz_gate', 'ry_gate']]]

In [62]:
ansatz_0 = population[0]
ansatz_0

[['rz_gate', 'rz_gate', 'phase', 'ry_gate'],
 ['hadamard', 'pauli_y', 'hadamard', 'pauli_z'],
 ['rz_gate', 'hadamard', 'hadamard', 'hadamard'],
 ['rx_gate', 'ctrl_0', 'trgt_0', 't']]

In [63]:
shuffled_ansatz = random.sample(ansatz_0, k=len(ansatz_0))
shuffled_ansatz

[['rx_gate', 'ctrl_0', 'trgt_0', 't'],
 ['hadamard', 'pauli_y', 'hadamard', 'pauli_z'],
 ['rz_gate', 'hadamard', 'hadamard', 'hadamard'],
 ['rz_gate', 'rz_gate', 'phase', 'ry_gate']]

In [50]:
len(population[1])

9

In [55]:
layer = mutate.generate_partially_entangled_layer(population[1][2])

In [56]:
layer

['pauli_x', 'ry_gate', 'ctrl_0', 'trgt_0']

In [51]:
population

[[['rz_gate', 'rz_gate', 'phase', 'ry_gate'],
  ['hadamard', 'pauli_y', 'hadamard', 'pauli_z'],
  ['rz_gate', 'hadamard', 'hadamard', 'hadamard'],
  ['rx_gate', 'ctrl_0', 'trgt_0', 't']],
 [['pauli_z', 'ctrl_0', 'trgt_0', 'pauli_y'],
  ['ctrl_0', 'trgt_0', 'ry_gate', 'pauli_z'],
  ['t', 'pauli_z', 'pauli_y', 'pauli_x'],
  ['hadamard', 't', 'pauli_x', 'pauli_z'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['t', 'ctrl_0', 'trgt_0', 'rx_gate'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['phase', 'ctrl_0', 'trgt_0', 'rz_gate'],
  ['rz_gate', 'rx_gate', 'rz_gate', 'ry_gate']]]

In [52]:
mutate._do(population)

([['rz_gate', 'rz_gate', 'phase', 'ry_gate'],
  ['pauli_y', 'pauli_y', 'hadamard', 'pauli_z'],
  ['rz_gate', 'hadamard', 'pauli_y', 'hadamard'],
  ['t', 'ctrl_0', 'trgt_0', 't']],
 [['ctrl_0', 'trgt_0', 'ry_gate', 'pauli_z'],
  ['rz_gate', 'rx_gate', 'rz_gate', 'ry_gate'],
  ['hadamard', 't', 'pauli_x', 'pauli_z'],
  ['t', 'pauli_z', 'pauli_y', 'pauli_x'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['rz_gate', 'rx_gate', 'rz_gate', 'ry_gate'],
  ['hadamard', 't', 'pauli_x', 'pauli_z'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1']])

In [57]:
class RepairAnsatz:
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        
    def _do(self, individual):
        new_size = random.randint(4, 8)
        selected_indexes = np.random.randint(0, high=len(individual), size=new_size)
    
        return [individual[index] for index in selected_indexes]

In [58]:
huge_ansatz = [['rz_gate', 'rz_gate', 'phase', 'ry_gate'],
  ['hadamard', 'pauli_y', 'hadamard', 'pauli_z'],
  ['rz_gate', 'hadamard', 'hadamard', 'hadamard'],
  ['rx_gate', 'ctrl_0', 'trgt_0', 't'],
  ['pauli_z', 'ctrl_0', 'trgt_0', 'pauli_y'],
  ['ctrl_0', 'trgt_0', 'ry_gate', 'pauli_z'],
  ['t', 'pauli_z', 'pauli_y', 'pauli_x'],
  ['hadamard', 't', 'pauli_x', 'pauli_z'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['t', 'ctrl_0', 'trgt_0', 'rx_gate'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['phase', 'ctrl_0', 'trgt_0', 'rz_gate'],
  ['rz_gate', 'rx_gate', 'rz_gate', 'ry_gate']]

In [59]:
repairer = RepairAnsatz(4)
repaired_ansatz = repairer._do(huge_ansatz)

In [60]:
repaired_ansatz

[['rz_gate', 'rx_gate', 'rz_gate', 'ry_gate'],
 ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
 ['rz_gate', 'hadamard', 'hadamard', 'hadamard'],
 ['hadamard', 'pauli_y', 'hadamard', 'pauli_z']]

In [82]:
import torch
from math import sqrt 
import itertools
from ansatz_simulation_class import AnsatzSimulation
from collections import Counter
import math
from torch import inner, tensor

class RemoveEquivalentAnsätze:

    def __init__(self, n_tests, threshold_value, n_qubits):
        self.n_tests = n_tests
        self.threshold_value = threshold_value
        self.n_qubits = n_qubits
        self.circuit_ansatz = AnsatzSimulation(self.n_qubits)
        
        

    def generate_random_parameters(self, ansatz):
        gates = list(itertools.chain.from_iterable(ansatz))
        gate_count = Counter(gates)
        n_params = gate_count['rx_gate'] + gate_count['ry_gate'] + gate_count['rz_gate']
        parameters = torch.rand(n_params)*math.pi

        return parameters

    def test_ansatz(self, ansatz, parameters, state_vector):
        return self.circuit_ansatz.simulate_circuit(input=state_vector, embedding_type='rx', ansatz_chromosome=ansatz, parameters=parameters, measure=False)
            
    # Check whether i calculate the inner product with just the real parts or should i include the imaginary ones as well
    def inner_product(self, state_vector_psi, state_vector_phi):
        return inner(state_vector_psi, state_vector_phi)

    def fidelity(self, state_vector_psi, state_vector_phi):
        return 1 - torch.einsum('i,i->i',state_vector_psi, state_vector_phi)**2


    def euclidian_distance(self, state_vector_psi, state_vector_phi):
        return sqrt(2 - 2*self.inner_product(state_vector_psi.real, state_vector_phi.real))

    # Decide which measure should be used in order to evaluate the

    def is_equal(self, offspring):
        ansatz_a, ansatz_b = offspring
        input_test = torch.rand(self.n_tests, self.n_qubits)
        params_a = self.generate_random_parameters(ansatz_a)
        params_b = self.generate_random_parameters(ansatz_b)
        
        output_a = [self.test_ansatz(ansatz_a, params_a, state_vector) for state_vector in input_test]
        output_b = [self.test_ansatz(ansatz_b, params_b, state_vector) for state_vector in input_test]
        output_pair = list(zip(output_a, output_b))
        
        fidelity_score = torch.stack([self.fidelity(state_psi, state_phi) for (state_psi, state_phi) in output_pair])
        
        score = torch.mean(fidelity_score)
        print(score)
        return self.threshold_value < score.item().real

In [83]:
discriminator = RemoveEquivalentAnsätze(n_tests=4, threshold_value=0.7, n_qubits=4)
result = discriminator.is_equal(population)
result

tensor(1.0096-0.0033j)


True